In [29]:
import string

class CombinedCipher:
    """
    Combined Vigenère + Affine Cipher Implementation.
    """

    def __init__(self):
        self.alphabet = string.ascii_uppercase
        self.M = 26

    def _gcd(self, a, b):
        while b:
            a, b = b, a % b
        return a

    def _prepare_text(self, text):
        return ''.join(c.upper() for c in text if c.isalpha())

    # --- Affine Stage ---

    def affine_encrypt(self, plaintext, a, b):
        """Encrypt using Affine cipher: C = (aP + b) mod 26"""
        if self._gcd(a, self.M) != 1:
            raise ValueError(f"'a' must be coprime with {self.M}. Got a={a}")

        plaintext = self._prepare_text(plaintext)
        ciphertext = []
        for char in plaintext:
            p = self.alphabet.index(char)
            c = (a * p + b) % self.M
            ciphertext.append(self.alphabet[c])
        return ''.join(ciphertext)

    def affine_decrypt(self, ciphertext, a, b):
        """Decrypt using Affine cipher: P = (a^-1 * (C - b)) mod 26"""
        if self._gcd(a, self.M) != 1:
            raise ValueError(f"'a' must be coprime with {self.M}. Got a={a}")

        ciphertext = self._prepare_text(ciphertext)
        a_inv = pow(a, -1, self.M)  # Modular multiplicative inverse
        plaintext = []
        for char in ciphertext:
            c = self.alphabet.index(char)
            p = (a_inv * (c - b)) % self.M
            plaintext.append(self.alphabet[p])
        return ''.join(plaintext)

    # --- Vigenère Stage ---

    def vigenere_encrypt(self, plaintext, key):
        """Encrypt using Vigenère cipher: C = (P + K_i) mod 26"""
        plaintext = self._prepare_text(plaintext)  # Convert text to uppercase and remove non-alphabet
        key = self._prepare_text(key)  # Ensure key is in uppercase and without non-alphabet characters
        ciphertext = []
        key_index = 0

        for char in plaintext:
            p = self.alphabet.index(char)
            k = self.alphabet.index(key[key_index % len(key)])  # Make sure key is within bounds
            c = (p + k) % self.M
            ciphertext.append(self.alphabet[c])
            key_index += 1
        return ''.join(ciphertext)

    def vigenere_decrypt(self, ciphertext, key):
        """Decrypt using Vigenère cipher: P = (C - K_i) mod 26"""
        ciphertext = self._prepare_text(ciphertext)  # Convert text to uppercase and remove non-alphabet
        key = self._prepare_text(key)  # Ensure key is in uppercase and without non-alphabet characters
        plaintext = []
        key_index = 0

        for char in ciphertext:
            c = self.alphabet.index(char)
            k = self.alphabet.index(key[key_index % len(key)])  # Make sure key is within bounds
            p = (c - k) % self.M
            plaintext.append(self.alphabet[p])
            key_index += 1
        return ''.join(plaintext)

    # --- Combined Functions ---

    def combined_encrypt(self, plaintext, vigenere_key, affine_a, affine_b, order="VA"):
        """
        Combined encryption: Vigenère + Affine (VA) or Affine + Vigenère (AV).
        """
        if order == "VA":
            temp = self.vigenere_encrypt(plaintext, vigenere_key)
            ciphertext = self.affine_encrypt(temp, affine_a, affine_b)
        else:  # AV
            temp = self.affine_encrypt(plaintext, affine_a, affine_b)
            ciphertext = self.vigenere_encrypt(temp, vigenere_key)
        return ciphertext

    def combined_decrypt(self, ciphertext, vigenere_key, affine_a, affine_b, order="VA"):
        """
        Combined Decryption: Reverses the encryption order.
        """
        if order == "VA":
            # Decrypt: Affine inverse -> Vigenère inverse
            temp = self.affine_decrypt(ciphertext, affine_a, affine_b)
            plaintext = self.vigenere_decrypt(temp, vigenere_key)
        else:  # AV
            # Decrypt: Vigenère inverse -> Affine inverse
            temp = self.vigenere_decrypt(ciphertext, vigenere_key)
            plaintext = self.affine_decrypt(temp, affine_a, affine_b)
        return plaintext


In [30]:
cipher = CombinedCipher()

# text and keys for testing
plaintext = "THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG"
vigenere_key = "KEYKEYKEY"
affine_a = 5
affine_b = 8

# Encrypt and Decrypt using Combined Cipher (VA order)
ciphertext = cipher.combined_encrypt(plaintext, vigenere_key, affine_a, affine_b, order="VA")
decrypted_text = cipher.combined_decrypt(ciphertext, vigenere_key, affine_a, affine_b, order="VA")

print("Plaintext: ", plaintext)
print("Ciphertext: ", ciphertext)
print("Decrypted Text: ", decrypted_text)


Plaintext:  THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG
Ciphertext:  XLSIYMQADNUETBQRVUOZKYDSNTHAFYBSNYG
Decrypted Text:  THEQUICKBROWNFOXJUMPSOVERTHELAZYDOG


In [31]:
# --- CipherBreaker class for frequency and known-plaintext attacks ---

import time
from collections import Counter

class CipherBreaker:
    """
    Advanced cipher breaking tool for the combined cipher.
    Implements frequency analysis and known-plaintext attacks.
    """

    def __init__(self, encryption_order="VA"):
        self.encryption_order = encryption_order
        self.alphabet = string.ascii_uppercase
        self.M = 26
        self.attack_log = []

        # English letter frequencies (in percentages)
        self.ENGLISH_FREQ = {
            'E': 12.70, 'T': 9.06, 'A': 8.17, 'O': 7.51, 'I': 6.97,
            'N': 6.75, 'S': 6.33, 'H': 6.09, 'R': 5.99, 'D': 4.25,
            'L': 4.03, 'C': 2.78, 'U': 2.76, 'M': 2.41, 'W': 2.36,
            'F': 2.23, 'G': 2.02, 'Y': 1.97, 'P': 1.93, 'B': 1.29,
            'V': 0.98, 'K': 0.77, 'J': 0.15, 'X': 0.15, 'Q': 0.10, 'Z': 0.07
        }

    def log(self, message):
        self.attack_log.append(message)
        print(message)

    def clear_log(self):
        self.attack_log = []

    def frequency_analysis(self, text):
        text = text.upper()
        total = sum(1 for c in text if c.isalpha())
        if total == 0: return {}
        freq = Counter(c for c in text if c.isalpha())
        return {letter: (count / total) * 100 for letter, count in freq.items()}

    def chi_squared(self, observed_freq):
        chi_sq = 0
        for letter in self.alphabet:
            expected = self.ENGLISH_FREQ.get(letter, 0)
            observed = observed_freq.get(letter, 0)
            if expected > 0:
                chi_sq += ((observed - expected) ** 2) / expected
        return chi_sq

    def index_of_coincidence(self, text):
        """Calculate Index of Coincidence for the text."""
        text = ''.join(c for c in text.upper() if c.isalpha())
        n = len(text)
        if n <= 1: return 0
        freq = Counter(text)
        ic = sum(f * (f - 1) for f in freq.values()) / (n * (n - 1))
        return ic

    def estimate_key_length(self, ciphertext, max_length=20):
        """Estimate Vigenère key length using Index of Coincidence."""
        self.log("\nESTIMATING VIGENÈRE KEY LENGTH (IC Analysis)")
        ciphertext = ''.join(c for c in ciphertext.upper() if c.isalpha())
        ic_scores = {}
        for key_len in range(1, max_length + 1):
            groups = [''] * key_len
            for i, char in enumerate(ciphertext):
                groups[i % key_len] += char
            avg_ic = sum(self.index_of_coincidence(g) for g in groups) / key_len
            ic_scores[key_len] = avg_ic

        sorted_keys = sorted(ic_scores.items(), key=lambda x: abs(x[1] - 0.067))
        best_key_length = sorted_keys[0][0]
        self.log(f"✓ Most likely key length: **{best_key_length}** (IC = {sorted_keys[0][1]:.4f})")
        return best_key_length, ic_scores


In [32]:
breaker = CipherBreaker(encryption_order="VA")

ciphertext = 'QEB NRFZH YOLTK CLU GRJMP LSBO QEB IXWV ALD'

freq_result = breaker.frequency_analysis(ciphertext)
print("Frequency Analysis: ", freq_result)

key_length, ic_scores = breaker.estimate_key_length(ciphertext)
print("Estimated Key Length: ", key_length)


Frequency Analysis:  {'Q': 5.714285714285714, 'E': 5.714285714285714, 'B': 8.571428571428571, 'N': 2.857142857142857, 'R': 5.714285714285714, 'F': 2.857142857142857, 'Z': 2.857142857142857, 'H': 2.857142857142857, 'Y': 2.857142857142857, 'O': 5.714285714285714, 'L': 11.428571428571429, 'T': 2.857142857142857, 'K': 2.857142857142857, 'C': 2.857142857142857, 'U': 2.857142857142857, 'G': 2.857142857142857, 'J': 2.857142857142857, 'M': 2.857142857142857, 'P': 2.857142857142857, 'S': 2.857142857142857, 'I': 2.857142857142857, 'X': 2.857142857142857, 'W': 2.857142857142857, 'V': 2.857142857142857, 'A': 2.857142857142857, 'D': 2.857142857142857}

ESTIMATING VIGENÈRE KEY LENGTH (IC Analysis)
✓ Most likely key length: **15** (IC = 0.0667)
Estimated Key Length:  15


In [33]:
# --- Run tests and collect success-rate and timing data ---
import time

def run_tests():
    cipher = CombinedCipher()

    # Test configurations (keys should be 10+ characters)
    test_cases = [
        {
            'name': 'Test Case 1: VA Order (Vigenère -> Affine)',
            'plaintext': 'THE QUICK BROWN FOX JUMPS OVER THE LAZY DOG',
            'vigenere_key': 'CRYPTOGRAPHYSECRET',
            'affine_a': 5,
            'affine_b': 8,
            'order': 'VA'
        },
        {
            'name': 'Test Case 2: AV Order (Affine -> Vigenère)',
            'plaintext': 'NETWORK AND INFORMATION SECURITY IS A CRITICAL FIELD',
            'vigenere_key': 'PROJECTNIS486KEY',
            'affine_a': 9,
            'affine_b': 15,
            'order': 'AV'
        }
    ]

    results_summary = []

    for test in test_cases:
        start_time = time.time()

        # Encrypt the plaintext
        ciphertext = cipher.combined_encrypt(test['plaintext'], test['vigenere_key'], test['affine_a'], test['affine_b'], test['order'])

        # Decrypt the ciphertext
        decrypted_text = cipher.combined_decrypt(ciphertext, test['vigenere_key'], test['affine_a'], test['affine_b'], test['order'])

        # Success Check (Comparing decrypted text to the original plaintext)
        success = decrypted_text.replace(" ", "") == test['plaintext'].replace(" ", "")

        elapsed_time = time.time() - start_time

        results_summary.append({
            'test_name': test['name'],
            'success': '✓ PASS' if success else '✗ FAIL',
            'time_taken': elapsed_time
        })

        print(f"{test['name']} - {'✓ PASS' if success else '✗ FAIL'} | Time taken: {elapsed_time:.4f} seconds")

    return results_summary

test_results = run_tests()


Test Case 1: VA Order (Vigenère -> Affine) - ✓ PASS | Time taken: 0.0001 seconds
Test Case 2: AV Order (Affine -> Vigenère) - ✓ PASS | Time taken: 0.0001 seconds
